 1. Data transformation (log, hot encoding, nan)
 
 2. Testing SVM regression models
 Applying log transformation really increases the accuracy.

Removing columns with more than 1000 NaNs gives better result than applying "mean" to them

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import Imputer

from sklearn import svm

import warnings
warnings.filterwarnings('ignore')

from subprocess import check_output
print(check_output(["ls", "input"]).decode("utf8"))

data_description.txt
sample_submission.csv
test.csv
train.csv



In [4]:
train = pd.read_csv('input/train.csv')
labels=train["SalePrice"]
test = pd.read_csv('input/test.csv')
data = pd.concat([train,test],ignore_index=True)
data = data.drop("SalePrice", 1)
ids = test["Id"]

In [5]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
# Count the number of rows in train
train.shape[0]


1460

In [7]:
# Count the number of rows in total
data.shape[0]

2919

In [8]:
# Count the number of NaNs each column has.
nans=pd.isnull(data).sum()
nans[nans>0]

Alley           2721
BsmtCond          82
BsmtExposure      82
BsmtFinSF1         1
BsmtFinSF2         1
BsmtFinType1      79
BsmtFinType2      80
BsmtFullBath       2
BsmtHalfBath       2
BsmtQual          81
BsmtUnfSF          1
Electrical         1
Exterior1st        1
Exterior2nd        1
Fence           2348
FireplaceQu     1420
Functional         2
GarageArea         1
GarageCars         1
GarageCond       159
GarageFinish     159
GarageQual       159
GarageType       157
GarageYrBlt      159
KitchenQual        1
LotFrontage      486
MSZoning           4
MasVnrArea        23
MasVnrType        24
MiscFeature     2814
PoolQC          2909
SaleType           1
TotalBsmtSF        1
Utilities          2
dtype: int64

In [9]:

data=data.drop("Id", 1)
data=data.drop("Alley", 1)
data=data.drop("Fence", 1)
data=data.drop("MiscFeature", 1)
data=data.drop("PoolQC", 1)
data=data.drop("FireplaceQu", 1)

In [10]:
# Count the column types
data.dtypes.value_counts()

object     38
int64      25
float64    11
dtype: int64

In [11]:
data.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,3,1Fam,TA,No,706.0,0.0,GLQ,...,WD,0,Pave,8,856.0,AllPub,0,2003,2003,2008
1,1262,0,0,3,1Fam,TA,Gd,978.0,0.0,ALQ,...,WD,0,Pave,6,1262.0,AllPub,298,1976,1976,2007
2,920,866,0,3,1Fam,TA,Mn,486.0,0.0,GLQ,...,WD,0,Pave,6,920.0,AllPub,0,2001,2002,2008
3,961,756,0,3,1Fam,Gd,No,216.0,0.0,ALQ,...,WD,0,Pave,7,756.0,AllPub,0,1915,1970,2006
4,1145,1053,0,4,1Fam,TA,Av,655.0,0.0,GLQ,...,WD,0,Pave,9,1145.0,AllPub,192,2000,2000,2008


In [12]:
all_columns = data.columns.values
non_categorical = ["LotFrontage", "LotArea", "MasVnrArea", "BsmtFinSF1", 
                   "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", 
                   "2ndFlrSF", "LowQualFinSF", "GrLivArea", "GarageArea", 
                   "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", 
                   "ScreenPorch","PoolArea", "MiscVal"]

categorical = [value for value in all_columns if value not in non_categorical]

In [13]:
# One Hot Encoding and nan transformation
data = pd.get_dummies(data)

imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
data = imp.fit_transform(data)

# Log transformation
data = np.log(data)
labels = np.log(labels)

# Change -inf to 0 again
data[data==-np.inf]=0

In [14]:
# Split traing and test
train = data[:1460]
test = data[1460:]

In [15]:
train.shape

(1460, 270)

In [16]:
labels.shape

(1460,)

In [17]:
clf = svm.SVR(kernel="rbf")

In [18]:
clf.fit(train,labels)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

In [19]:
test_labels = clf.predict(test)

In [20]:
test_labels.shape

(1459,)

In [21]:
print(test_labels)

[11.81624666 12.23470153 12.14372613 ... 12.08087383 11.60034954
 12.44674734]


In [22]:
price_label = np.exp(test_labels)
price_labels = price_label.reshape(-1,)

In [23]:
print(price_labels)

[135434.9400385  205808.52634403 187911.45262461 ... 176464.29286235
 109135.94024166 254421.42909117]


In [24]:
sub = pd.DataFrame({
        "SalePrice": price_labels
    })

In [25]:
sub.head()

,SalePrice
0,135434.940039
1,205808.526344
2,187911.452625
3,202696.372009
4,179084.218149


In [29]:
test_data = pd.read_csv('input/test.csv')

In [30]:
result =  test_data.append(sub)

In [31]:
result['SalePrice'] = sub['SalePrice']

In [32]:
result['SalePrice'].head()

0    135434.940039
1    205808.526344
2    187911.452625
3    202696.372009
4    179084.218149
Name: SalePrice, dtype: float64

In [33]:
result.to_csv("sample_submission.csv", index=False)